<a href="https://colab.research.google.com/github/pulipulichen/colab-File-Browser/blob/main/colab/colab-File-Browser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **colab-File-Browser**

Instruction: https://sites.google.com/view/howtousecolab-colab-20250202-2


# Script

## Configurations for script

In [1]:
PROJECT_OWNER="pulipulichen"
PROJECT_NAME="colab-File-Browser"

FORCE_RUN_IN_FOREGROUND=True

### Testing Instruction

1. `Runtime` > `Run all` (Ctrl + F9)
2. Wait for the Cloudflare URL to be accessible.

## Core Tools

### Install Packages that need restart the session.

In [2]:
# Install packages that will trigger a session restart first.
# !pip install CherryPy==18.8.0

### curl_local_url()

In [3]:
import os
import time
import subprocess

def curl_local_url(port):
  while True:
    try:
      output = subprocess.check_output(['curl', '-s', 'http://127.0.0.1:' + str(port) + '/']).decode('utf-8')
      print("Success to connect http://127.0.0.1:" + str(port) + ' ' + get_current_datestring())
      break  # Break out of the loop if successful
    except subprocess.CalledProcessError:
      # print("Check http://127.0.0.1:" + str(port) + "/ failed. Retrying in 5 seconds...")
      time.sleep(5)
      continue  # Retry in case of failure

### wait_for_cloudflare()

In [4]:
import os
import time

def wait_for_cloudflare():
  file_path = f'/content/docker-app/{PROJECT_NAME}/.cloudflare.url'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")


### wait_for_docker_web_ready()

In [5]:
import os
import time

def wait_for_docker_web_ready():
  print('\nWaiting for docker web ready...' + get_current_datestring())
  file_path = f'/content/docker-app/{PROJECT_NAME}/.docker-web.ready'

  while not os.path.exists(file_path):
    time.sleep(3)  # Check every 1 second

  # File found, read and print its contents
  # with open(file_path, 'r') as file:
  #   content = file.read()
  #   print(f"Public URL: {content}")

  time.sleep(10)

### keep_waiting()

In [6]:
import time

def keep_waiting():
  while True:
    time.sleep(1)  # Check every 1 second


### mountGDrive()

In [7]:
import os
from subprocess import getoutput
from google.colab import drive
import subprocess

try:
    GDRIVE_PATH
except NameError:
    GDRIVE_PATH = "/colab/" + PROJECT_NAME

def mountGDrive():
  if os.path.exists("/google-drive/MyDrive"):
    return True

  if not os.path.exists("/google-drive"):
    drive.mount('/google-drive')  # access drive

  TEMP_GDRIVE_PATH = GDRIVE_PATH

  if not TEMP_GDRIVE_PATH.startswith("/"):
    TEMP_GDRIVE_PATH = "/" + TEMP_GDRIVE_PATH
  if not TEMP_GDRIVE_PATH.endswith("/"):
    TEMP_GDRIVE_PATH = TEMP_GDRIVE_PATH + "/"

  folder_path = "/google-drive/MyDrive/docker-app" + TEMP_GDRIVE_PATH
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  target_folder_path = "/content/docker-app/"
  if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

  command = ["ln", "-s", folder_path, target_folder_path]
  subprocess.run(command, check=True)

### clearDockerData()

In [8]:
import os
from subprocess import getoutput
import subprocess

def clearDockerData():
  stop_udocker()
  command = ["rm", "-rf", "/content/docker-app/" + PROJECT_NAME]
  subprocess.run(command, check=True)


### get_config_from_yaml()

In [9]:
import yaml
import time

def get_config_from_yaml():
  yaml_file_path = f"./.docker-app/{PROJECT_NAME}/app-build/docker-compose-template.yml"

  with open(yaml_file_path, "r") as file:
    yaml_data = yaml.safe_load(file)
  # print(yaml_data)

  # Assuming there is only one service named "app" in your YAML
  webapp_ports = yaml_data.get("services", {}).get("app", {}).get("ports", [])
  WEBAPP_PORT = None

  for port_mapping in webapp_ports:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(port_mapping, str) and ":" in port_mapping:
        # print(port_mapping)
        host_port, container_port = port_mapping.split(":")
        WEBAPP_PORT = host_port

        if WEBAPP_PORT.endswith('"'):
          WEBAPP_PORT = WEBAPP_PORT[:-1]  # Remove the last character

        break

  # ============
  environments = yaml_data.get("services", {}).get("app", {}).get("environment", [])

  # ============
  RUN_COMMAND = yaml_data.get("services", {}).get("app", {}).get("command", [])
  if RUN_COMMAND is not None:
    RUN_COMMAND = " ".join(RUN_COMMAND)

  # ============

  volumes = yaml_data.get("services", {}).get("app", {}).get("volumes", [])
  LOCAL_VOLUMN_PATH = '/data'

  for volumes_mapping in volumes:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(volumes_mapping, str) and ":" in port_mapping:
        # print(port_mapping)
        host_path, container_path = volumes_mapping.split(":")
        LOCAL_VOLUMN_PATH = container_path
        break

  # ============
  deploy = yaml_data.get("services", {}).get("app", {}).get("deploy", [])
  GPU_ENABLE = False
  if 'resources' in deploy:
    GPU_ENABLE = True

  # ============
  app = yaml_data.get("services", {}).get("app", {})
  # print(app)
  IMAGE_NAME = app['image']

  return {
    "WEBAPP_PORT": WEBAPP_PORT,
    "LOCAL_VOLUMN_PATH": LOCAL_VOLUMN_PATH,
    "IMAGE_NAME": IMAGE_NAME,
    "RUN_COMMAND": RUN_COMMAND,
    "GPU_ENABLE": GPU_ENABLE
  }


#### get_environments_from_yaml()

In [10]:
def get_environments_from_yaml(environments, key, default_value = None):

  for env_mapping in environments:
      # Check if the mapping is in the format "HOST_PORT:CONTAINER_PORT"
      if isinstance(env_mapping, str) and key + "=" in env_mapping:
        # print(port_mapping)
        key, value = env_mapping.split("=")
        default_value = value
        break

  return default_value

### get_current_datestring()

In [11]:
from datetime import datetime

def get_current_datestring():
  return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

### show_interval_time()

In [12]:
from datetime import datetime
import math

def show_interval_time():
  ready_time = datetime.now()

  # Calculate the difference in minutes
  interval_minutes = math.ceil((ready_time - START_TIME).total_seconds() / 60)
  print(f"{interval_minutes} minutes spent in the startup process..." + get_current_datestring())

  # print('\nPreparing link... ' + ready_time.strftime("%Y-%m-%d %H:%M:%S"))


## udocker

### stop_udocker()

In [13]:
def stop_udocker():
  #!udocker --allow-root ps
  print('Stop udocker...' + get_current_datestring())
  !udocker --allow-root ps | awk 'NR > 1 {print $1}' | xargs -I {} udocker --allow-root rm {}

  print('Kill Java...' + get_current_datestring())
  !pgrep java && pkill java && sleep 5


### run_udocker()

In [14]:
def run_udocker():
  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()

  # !rm -f /content/docker-app/$PROJECT_NAME/.docker-web.ready || true
  # !rm -f /content/docker-app/$PROJECT_NAME/.cloudflare.url || true
  ![ -e /content/docker-app/$PROJECT_NAME/.cloudflare.url ] && rm /content/docker-app/$PROJECT_NAME/.cloudflare.url
  ![ -e /content/docker-app/$PROJECT_NAME/.docker-web.ready ] && rm /content/docker-app/$PROJECT_NAME/.docker-web.ready

  run_udocker_in_foreground()

  # RUN_IN_BACKGROUND = True
  # if YAML_CONFIG["WEBAPP_PORT"] is None:
  #   RUN_IN_BACKGROUND = False



  # # For testing purposes
  # if FORCE_RUN_IN_FOREGROUND is True:
  #   RUN_IN_BACKGROUND = False

  # if RUN_IN_BACKGROUND is False:
  #   run_udocker_in_foreground()
  # else:
  #   run_udocker_in_background()


### run_udocker_in_foreground()

In [15]:
# 會觸發Session Restart
# !pip install CherryPy==18.8.0
# import cherrypy

def run_udocker_in_foreground():

  # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
  YAML_CONFIG = get_config_from_yaml()
  # cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port' : int(YAML_CONFIG["WEBAPP_PORT"])})

  print('\nRun container in foreground...' + get_current_datestring())

  # print(LOCAL_VOLUMN_PATH)
  # print(RUN_COMMAND)

  !rm -rf /content/docker-app/$PROJECT_NAME
  !mkdir -p /content/docker-app/$PROJECT_NAME

  # asyncio.run(setup_cloudflared())

  # =======

  # show_interval_time()

  # =======

  udocker_command = get_udocker_run_command()

  run_script = """#!/bin/bash

waitForConntaction() {
  port="$1"
  sleep 3
  while true; do
    # echo "http://127.0.0.1:$port"
    if curl -sSf "http://127.0.0.1:$port" >/dev/null 2>&1; then
      # echo "Connection successful."
      break
    else
      #echo "Connection failed. Retrying in 5 seconds..."
      sleep 10
    fi
  done
}

""" + udocker_command + """ &

WEPAPP_PORT=""" + YAML_CONFIG["WEBAPP_PORT"] + """

waitForConntaction $WEPAPP_PORT

while [ ! -f /content/docker-app/""" + PROJECT_NAME +"""/.docker-web.ready ]; do
    sleep 1  # Wait for 1 second before checking again
done

setupCloudflare() {
  file="/tmp/cloudflared"
  if [ ! -f "$file" ]; then
    wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O "${file}"
    chmod a+x "${file}"
  fi
}

runCloudflare() {
  port="$1"
  file_path="$2"

  rm -rf "${file_path}"
  #nohup /tmp/.cloudflared --url "http://127.0.0.1:${port}" > "${file_path}" 2>&1 &
  /tmp/cloudflared --url "http://127.0.0.1:${port}" > "${file_path}" 2>&1 &
}

getCloudflarePublicURL() {
  setupCloudflare

  port="$1"

    # File path
  file_path="/tmp/cloudflared.out"

  runCloudflare "$port" "${file_path}" &

  sleep 5

  # Retry logic
  max_retries=5  # Set max retries to avoid infinite loops
  retry_count=0

  while [[ $retry_count -lt $max_retries ]]; do
    # Extracting the URL using grep and awk
    url=$(grep -o 'https://[^ ]*\.trycloudflare\.com' "$file_path" | awk '/https:\/\/[^ ]*\.trycloudflare\.com/{print; exit}')

    if [[ -n "$url" ]]; then
      echo "$url"
      return 0
    fi

    # If URL is empty, wait 5 seconds and retry
    echo "URL not found, retrying in 5 seconds... (Attempt $((retry_count + 1))/$max_retries)"
    sleep 5
    ((retry_count++))
  done

  echo "Failed to retrieve Cloudflare URL after $max_retries attempts." >&2
  return 1
}

getCloudflarePublicURL """ + YAML_CONFIG["WEBAPP_PORT"] + """ > "/tmp/.cloudflare.url"

echo "\n================================================================"
echo "Public URL:"
cat "/tmp/.cloudflare.url"
echo "================================================================"
"""

  setup_script_path = "/content/.docker-app/setup_script.sh"
  with open(setup_script_path, "w") as file:
    file.write(run_script)

  !chmod +x $setup_script_path
  !bash $setup_script_path

### run_udocker_in_background()

In [16]:
# 會觸發Session Restart
# !pip install CherryPy==18.8.0

# import cherrypy

# def run_udocker_in_background():


#   stop_udocker()

#   !mkdir -p /content/docker-app/$PROJECT_NAME

#   # WEBAPP_PORT, LOCAL_VOLUMN_PATH, IMAGE_NAME, RUN_COMMAND = get_config_from_yaml()
#   YAML_CONFIG = get_config_from_yaml()

#   !rm -rf ./*nohup.out
#   !rm -rf ./docker-app/$PROJECT_NAME/.cloudflare.url

#   print('\nRun container in background...' + get_current_datestring())

#   ![ -e /content/.docker-app/udocker.sh ] && rm /content/.docker-app/udocker.sh
#   udocker_command = get_udocker_run_command()
#   !echo "nohup $udocker_command > /content/.docker-app/.nohup.out 2>&1 &" >> /content/.docker-app/udocker.sh
#   !chmod +x /content/.docker-app/udocker.sh
#   !bash /content/.docker-app/udocker.sh

#   cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port' : int(YAML_CONFIG["WEBAPP_PORT"])})
#   # wait_for_cloudflare()
#   curl_local_url(YAML_CONFIG["WEBAPP_PORT"])
#   wait_for_docker_web_ready()

#   # =======

#   show_interval_time()

#   # =======

#   # setup_cloudflared()

#   keep_waiting()

### get_udocker_run_command()

In [17]:
!pip install udocker==1.3.10
!udocker --allow-root install
import re

def get_udocker_run_command():
  YAML_CONFIG = get_config_from_yaml()

  command = ''

  IMAGE_NAME = YAML_CONFIG["IMAGE_NAME"]
  NAME_SPACE = re.sub(r'[/\-:.]', '_', IMAGE_NAME)

  print('\nPulling the image ' + IMAGE_NAME + '...' + get_current_datestring())
  !udocker --allow-root pull $IMAGE_NAME

  print('\nCreate the name space for ' + IMAGE_NAME + ' ...' + get_current_datestring())
  !udocker --allow-root create --name=$NAME_SPACE $IMAGE_NAME

  if YAML_CONFIG["GPU_ENABLE"] == True:

    print('\nSetup nvidia for the name space sdw...' + get_current_datestring())
    !udocker --allow-root setup --nvidia --force $NAME_SPACE

  command = "udocker --allow-root run " + \
    "-p " + YAML_CONFIG["WEBAPP_PORT"] + ":" + YAML_CONFIG["WEBAPP_PORT"] + \
    " " + \
    "--volume=/content/docker-app/" + PROJECT_NAME + ":" + YAML_CONFIG["LOCAL_VOLUMN_PATH"] + \
    " " + \
    NAME_SPACE

  print(command)

  return command

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 5.3 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.10
Info: searching for udockertools >= 1.2.10
Info: installing udockertools 1.2.10
Info: installation of udockertools successful


### setup_cloudflared()

In [18]:
import time
import re

def setup_cloudflared():
  YAML_CONFIG = get_config_from_yaml()

  print('\nWaiting for server ready... ' + get_current_datestring())
  curl_local_url(YAML_CONFIG["WEBAPP_PORT"])
  wait_for_docker_web_ready()

  if os.path.isfile('/content/.docker-app/.cloudflared') is False:
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/.docker-app/.cloudflared
    !chmod +x /content/.docker-app/.cloudflared

  #!cat /content/docker-app/docker-web-Apache-Solr/solrconfig.xml | grep dc.publisher
  cloudflared_command = '/content/.docker-app/.cloudflared --url "http://127.0.0.1:' + YAML_CONFIG["WEBAPP_PORT"] + '"'


  !echo "nohup $cloudflared_command > /content/.docker-app/.cloudflared.nohup.out 2>&1 &" >> /content/.docker-app/cloudflared.sh
  !chmod +x /content/.docker-app/cloudflared.sh
  !bash /content/.docker-app/cloudflared.sh

  time.sleep(10)

  file_path = "/content/.docker-app/.cloudflared.nohup.out"
  cloudflare_url = None

  while cloudflare_url is None:
      try:
          with open(file_path, "r") as file:
              content = file.read()
              match = re.search(r"https?://[\w.-]+\.trycloudflare\.com", content)
              if match:
                  cloudflare_url = match.group(0)
                  # print(f"Found Cloudflare URL: {cloudflare_url} " + get_current_datestring())
                  break
      except FileNotFoundError:
          print("File not found. Retrying in 10 seconds...")
      except Exception as e:
          print(f"Error reading file: {e}")

      time.sleep(10)

  print('\n============================================================================')
  show_interval_time()
  print('Public URL: ' + cloudflare_url)
  # print('')
  # print('You still need to wait for the subsequent execution to succeed before the URL displays the correct content.')
  print('============================================================================')

### setup_git_project()

In [19]:
def setup_git_project():
  if os.path.isdir('/content/.docker-app/') is False:
    !mkdir -p ./.docker-app

  if os.path.isdir('/content/.docker-app/' + PROJECT_NAME) is False:
    print('Try to initialize project...' + get_current_datestring())

    %cd /content/.docker-app
    !git clone "https://github.com/{PROJECT_OWNER}/{PROJECT_NAME}.git"
    !git config --global pull.rebase false
    !git config --global user.email "blog@pulipuli.info"
    !git config --global user.name "Pulipuli Chen"
  else:
    print('Try to update project...' + get_current_datestring())

    %cd /content/.docker-app/$PROJECT_NAME
    !git reset --hard
    !git pull --force

  %cd /content/

## main() (must be the last)

In [20]:
%cd /content
if os.path.isdir('/content/sample_data'):
  !rm -rf ./sample_data*

# ==================

from datetime import datetime
START_TIME = datetime.now()

def main():
  setup_git_project()

  START_TIME = datetime.now()
  print('\nPreparing environment... ' + START_TIME.strftime("%Y-%m-%d %H:%M:%S"))

  # mountGDrive() # 開啟Google Drive掛載功能
  run_udocker()

/content


In [21]:
# For Testing purposes

#clearDockerData()

# **Runtime**

In [22]:
# mountGDrive() # 開啟Google Drive掛載功能
main()

Try to initialize project...2025-02-04 09:00:37
/content/.docker-app
Cloning into 'colab-File-Browser'...
remote: Enumerating objects: 819, done.
remote: Counting objects: 100% (819/819), done.
remote: Compressing objects: 100% (708/708), done.
remote: Total 819 (delta 160), reused 709 (delta 95), pack-reused 0 (from 0)
Receiving objects: 100% (819/819), 2.49 MiB | 7.50 MiB/s, done.
Resolving deltas: 100% (160/160), done.
/content

Preparing environment... 2025-02-04 09:00:38

Run container in foreground...2025-02-04 09:00:38

Pulling the image pudding/docker-web:colab-file-browser-20250203.210626...2025-02-04 09:00:39
Info: downloading layer sha256:81463de3d3e40def8a8deebfdb5df4131a0c386dcfc1f5b2ba35ac29528ab486
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:22a000724d336b2928d092eb5c6706338a1ec497f1c65457557fefb5d8a3a79b
Info: downloading layer sha256:765ac7573f365d1e418b43cdb31f052323c85faa2a0dd0ddc4d5c1

KeyboardInterrupt: 